In [8]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tltk
tltk.pos_load()
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
%matplotlib inline

In [16]:
# Path to the preprocessed data
best_processed_path = 'w_review_train.csv'

In [17]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '@': 33, 'A': 34, 'B': 35, 'C': 36, 'D': 37, 'E': 38, 'F': 39, 'G': 40, 'H': 41, 'I': 42, 'J': 43, 'K': 44, 'L': 45, 'M': 46, 'N': 47, 'O': 48, 'P': 49, 'Q': 50, 'R': 51, 'S': 52, 'T': 53, 'U': 54, 'V': 55, 'W': 56, 'X': 57, 'Y': 58, 'Z': 59, '[': 60, '\\': 61, ']': 62, '^': 63, '_': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'other': 80, 'p': 81, 'q': 82, 'r': 83, 's': 84, 't': 85, 'u': 86, 'v': 87, 'w': 88, 'x': 89, 'y': 90, 'z': 91, '}': 92, '~': 93, 'ก': 94, 'ข': 95, 'ฃ': 96, 'ค': 97, 'ฅ': 98, 'ฆ': 99, 'ง': 100, 'จ': 101, 'ฉ': 102, 'ช': 103, 'ซ': 104, 'ฌ': 105, 'ญ': 106, 'ฎ': 107, 'ฏ': 108, 'ฐ': 109, 'ฑ'

In [18]:
from keras.models import Model
from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
from keras.layers import Input, GRU, Bidirectional
from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)ฅ
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model
    

In [20]:
weight_path_my_best_model2='/data/model_best.h5'
my_best_model2 = get_my_best_model()
my_best_model2.load_weights(weight_path_my_best_model2)
#my_best_model2.make_predict_function()
my_best_model2.summary()

OSError: Unable to open file (Unable to open file: name = '/data/model_best.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [21]:
df = pd.read_csv(best_processed_path, sep=';', header=None)

In [ ]:
# df

In [ ]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [22]:
word_test = [x for x in df[0][1]]
print(''.join(word_test))

อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก "ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 
อาหารที่เราแนะนำคือไก่เบตง (คล้ายๆไก่แช่เหล้า) เสริฟพร้อมกับหอมเจียว และน้ำจิ้มน้ำพริกเผาสูตรเด็ดของทางร้าน 
เมนูข้าวผัดหนำเลียบก็อร่อยค่ะ ชอบมากๆ


In [23]:
word_test_df2 = pd.DataFrame(word_test)
#print(word_test_df2)

n_pad = 21
n_pad_2 = int((n_pad - 1)/2)
pad = [{0: ' '}]
df_pad = pd.DataFrame(pad * n_pad_2)
#print(df_pad)

word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
#print(word_test_df2[0][:21])
word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))

In [24]:
df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

# convert pandas dataframe to numpy array to feed to the model
x_char = df_with_context[char_row].as_matrix()

NameError: name 'create_n_gram_df' is not defined

In [ ]:
y_pred = my_best_model2.predict(x_char)

prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

In [ ]:
print(len(y_pred))
print(len(word_test))
word_token = []
word_tmp = ''
for i in range(len(y_pred)):
    if(y_pred[i] == 1):
        word_token.append(word_tmp)
        word_tmp = ''
    word_tmp += word_test[i]
else:
    word_token.append(word_tmp)
word_token = word_token[1:]
print(word_token)

In [ ]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    return word_token

In [ ]:
def mergeLabel(review, rating):
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))
    label = [int(rating)] * len(cleaned)
    training_data = list(zip(cleaned, label))
    return training_data

In [25]:
mergeLabel(df[0][1], df[1][1])

NameError: name 'mergeLabel' is not defined

In [26]:
demo = df[:][:10000]
demo

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [27]:
train = []
for i in range(len(demo)):
    train.append((demo[0][i], demo[1][i]))

In [478]:
# train

In [ ]:
import re
def remove_htmlTags(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
    
def isValidWord(text):
    dic = {text[0]:1}
    for c in text[1:]:
        if c not in dic:
            return True
        else:
            dic[c] += 1
    if dic[text[0]] > 1:
        return False
    else:
        return True
        
def tokenize(review):
    
#     tags = tltk.nlp.pos_tag(review.replace("\n", ""))
#     noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
#     cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
#     for i in range(len(cleaned)):
#         cleaned[i] = cleaned[i].replace("\n", "")
#         cleaned[i] = cleaned[i].replace("(", "")
#         cleaned[i] = cleaned[i].replace(")", "")
#     finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))

    # Filter parts of speech of text
    tags = tltk.nlp.pos_tag(remove_htmlTags(review.replace("\n", "")))
    finalWord = []
    filteredTags = {'NOUN', 'VERB', 'ADJ','ADV', 'AUX', 'PART'}
    for sentence in tags:
        for word, tag in sentence:
            if tag in filteredTags:
                if len(word) > 1 and isValidWord(word):
                    finalWord.append(remove_htmlTags(word))
    return finalWord

# tokenize(train[0][0])
# train[0][0]

In [22]:
# from itertools import chain
# vocabulary = set(chain(*[tokenize(i[0].lower()) for i in train]))

In [20]:
# vocabulary
tokenize(train[999][0])

['ไม่',
 'รู้จัก',
 'ร้าน',
 'หรอก',
 'ครับ',
 'ไม่ค่อย',
 'ได้',
 'มา',
 'แถว',
 'พี่',
 'สาว',
 'แฟน',
 'ชวน',
 'มา',
 'กิน',
 'เปิด',
 'ดู',
 'เว็บ',
 'หา',
 'แผนที่',
 'ร้าน',
 'เห็น',
 'รูป',
 'ร้าน',
 'สวย',
 'มาก',
 'ครับ',
 'มี',
 'โต๊ะ',
 'ริม',
 'น้ำ',
 'ไม่',
 'มาก',
 'ต้อง',
 'รีบ',
 'จอง',
 'ห้อง',
 'แอร์',
 'มี',
 'เหมือนกัน',
 'ด้าน',
 'นอก',
 'บรรยากาศ',
 'ดี',
 'ครับ',
 'อาหาร',
 'ราคา',
 'ค่อนข้าง',
 'สูง',
 'ปริมาณ',
 'เยอะ',
 'รสชาติ',
 'ดี',
 'ครับ',
 'กุ้ง',
 'หอบ',
 'ปู',
 'ปลา',
 'สด',
 'ดี',
 'มี',
 'คุณภาพ',
 'พนักงานบริิการดีครับ',
 'มี',
 'อารมณ์ขัน',
 'ด้วย',
 'อาหาร',
 'จะ',
 'นาน',
 'ไป',
 'บ้าง',
 'พนักงาน',
 'ไม่',
 'ชัก',
 'สีห',
 'ร้า',
 'เวลา',
 'โดน',
 'ทวงถาม',
 'บ่อยๆ']

In [23]:
from IPython.display import clear_output
corpus = []
for i in range(len(train)):
    print(str(i+1) + "/" + str(len(train)))
    corpus.append(tokenize(train[i][0]))

corpus
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

1/4000
2/4000
3/4000
4/4000
5/4000
6/4000
7/4000
8/4000
9/4000
10/4000
11/4000
12/4000
13/4000
14/4000
15/4000
16/4000
17/4000
18/4000
19/4000
20/4000
21/4000
22/4000
23/4000
24/4000
25/4000
26/4000
27/4000
28/4000
29/4000
30/4000
31/4000
32/4000
33/4000
34/4000
35/4000
36/4000
37/4000
38/4000
39/4000
40/4000
41/4000
42/4000
43/4000
44/4000
45/4000
46/4000
47/4000
48/4000
49/4000
50/4000
51/4000
52/4000
53/4000
54/4000
55/4000
56/4000
57/4000
58/4000
59/4000
60/4000
61/4000
62/4000
63/4000
64/4000
65/4000
66/4000
67/4000
68/4000
69/4000
70/4000
71/4000
72/4000
73/4000
74/4000
75/4000
76/4000
77/4000
78/4000
79/4000
80/4000
81/4000
82/4000
83/4000
84/4000
85/4000
86/4000
87/4000
88/4000
89/4000
90/4000
91/4000
92/4000
93/4000
94/4000
95/4000
96/4000
97/4000
98/4000
99/4000
100/4000
101/4000
102/4000
103/4000
104/4000
105/4000
106/4000
107/4000
108/4000
109/4000
110/4000
111/4000
112/4000
113/4000
114/4000
115/4000
116/4000
117/4000
118/4000
119/4000
120/4000
121/4000
122/4000
123/4000
1

924/4000
925/4000
926/4000
927/4000
928/4000
929/4000
930/4000
931/4000
932/4000
933/4000
934/4000
935/4000
936/4000
937/4000
938/4000
939/4000
940/4000
941/4000
942/4000
943/4000
944/4000
945/4000
946/4000
947/4000
948/4000
949/4000
950/4000
951/4000
952/4000
953/4000
954/4000
955/4000
956/4000
957/4000
958/4000
959/4000
960/4000
961/4000
962/4000
963/4000
964/4000
965/4000
966/4000
967/4000
968/4000
969/4000
970/4000
971/4000
972/4000
973/4000
974/4000
975/4000
976/4000
977/4000
978/4000
979/4000
980/4000
981/4000
982/4000
983/4000
984/4000
985/4000
986/4000
987/4000
988/4000
989/4000
990/4000
991/4000
992/4000
993/4000
994/4000
995/4000
996/4000
997/4000
998/4000
999/4000
1000/4000
1001/4000
1002/4000
1003/4000
1004/4000
1005/4000
1006/4000
1007/4000
1008/4000
1009/4000
1010/4000
1011/4000
1012/4000
1013/4000
1014/4000
1015/4000
1016/4000
1017/4000
1018/4000
1019/4000
1020/4000
1021/4000
1022/4000
1023/4000
1024/4000
1025/4000
1026/4000
1027/4000
1028/4000
1029/4000
1030/4000
1031/4

1751/4000
1752/4000
1753/4000
1754/4000
1755/4000
1756/4000
1757/4000
1758/4000
1759/4000
1760/4000
1761/4000
1762/4000
1763/4000
1764/4000
1765/4000
1766/4000
1767/4000
1768/4000
1769/4000
1770/4000
1771/4000
1772/4000
1773/4000
1774/4000
1775/4000
1776/4000
1777/4000
1778/4000
1779/4000
1780/4000
1781/4000
1782/4000
1783/4000
1784/4000
1785/4000
1786/4000
1787/4000
1788/4000
1789/4000
1790/4000
1791/4000
1792/4000
1793/4000
1794/4000
1795/4000
1796/4000
1797/4000
1798/4000
1799/4000
1800/4000
1801/4000
1802/4000
1803/4000
1804/4000
1805/4000
1806/4000
1807/4000
1808/4000
1809/4000
1810/4000
1811/4000
1812/4000
1813/4000
1814/4000
1815/4000
1816/4000
1817/4000
1818/4000
1819/4000
1820/4000
1821/4000
1822/4000
1823/4000
1824/4000
1825/4000
1826/4000
1827/4000
1828/4000
1829/4000
1830/4000
1831/4000
1832/4000
1833/4000
1834/4000
1835/4000
1836/4000
1837/4000
1838/4000
1839/4000
1840/4000
1841/4000
1842/4000
1843/4000
1844/4000
1845/4000
1846/4000
1847/4000
1848/4000
1849/4000
1850/4000


2571/4000
2572/4000
2573/4000
2574/4000
2575/4000
2576/4000
2577/4000
2578/4000
2579/4000
2580/4000
2581/4000
2582/4000
2583/4000
2584/4000
2585/4000
2586/4000
2587/4000
2588/4000
2589/4000
2590/4000
2591/4000
2592/4000
2593/4000
2594/4000
2595/4000
2596/4000
2597/4000
2598/4000
2599/4000
2600/4000
2601/4000
2602/4000
2603/4000
2604/4000
2605/4000
2606/4000
2607/4000
2608/4000
2609/4000
2610/4000
2611/4000
2612/4000
2613/4000
2614/4000
2615/4000
2616/4000
2617/4000
2618/4000
2619/4000
2620/4000
2621/4000
2622/4000
2623/4000
2624/4000
2625/4000
2626/4000
2627/4000
2628/4000
2629/4000
2630/4000
2631/4000
2632/4000
2633/4000
2634/4000
2635/4000
2636/4000
2637/4000
2638/4000
2639/4000
2640/4000
2641/4000
2642/4000
2643/4000
2644/4000
2645/4000
2646/4000
2647/4000
2648/4000
2649/4000
2650/4000
2651/4000
2652/4000
2653/4000
2654/4000
2655/4000
2656/4000
2657/4000
2658/4000
2659/4000
2660/4000
2661/4000
2662/4000
2663/4000
2664/4000
2665/4000
2666/4000
2667/4000
2668/4000
2669/4000
2670/4000


3391/4000
3392/4000
3393/4000
3394/4000
3395/4000
3396/4000
3397/4000
3398/4000
3399/4000
3400/4000
3401/4000
3402/4000
3403/4000
3404/4000
3405/4000
3406/4000
3407/4000
3408/4000
3409/4000
3410/4000
3411/4000
3412/4000
3413/4000
3414/4000
3415/4000
3416/4000
3417/4000
3418/4000
3419/4000
3420/4000
3421/4000
3422/4000
3423/4000
3424/4000
3425/4000
3426/4000
3427/4000
3428/4000
3429/4000
3430/4000
3431/4000
3432/4000
3433/4000
3434/4000
3435/4000
3436/4000
3437/4000
3438/4000
3439/4000
3440/4000
3441/4000
3442/4000
3443/4000
3444/4000
3445/4000
3446/4000
3447/4000
3448/4000
3449/4000
3450/4000
3451/4000
3452/4000
3453/4000
3454/4000
3455/4000
3456/4000
3457/4000
3458/4000
3459/4000
3460/4000
3461/4000
3462/4000
3463/4000
3464/4000
3465/4000
3466/4000
3467/4000
3468/4000
3469/4000
3470/4000
3471/4000
3472/4000
3473/4000
3474/4000
3475/4000
3476/4000
3477/4000
3478/4000
3479/4000
3480/4000
3481/4000
3482/4000
3483/4000
3484/4000
3485/4000
3486/4000
3487/4000
3488/4000
3489/4000
3490/4000


In [70]:

import pickle


# pickle_out = open("corpus.pickle","wb")
# pickle.dump(corpus, pickle_out)
# pickle_out.close()


In [71]:
pickle_in = open("corpus1.pickle","rb")
corpus1 = pickle.load(pickle_in)
len(corpus1)
# corpus

5876

In [72]:
pickle_in = open("corpus2.pickle","rb")
corpus2 = pickle.load(pickle_in)
len(corpus2)

4124

In [73]:
# corpus
corpus = corpus1 + corpus2

newcor = corpus[:9000]
test = corpus[9000:]
# tltk.nlp.pos_tag(train[0][0])
len(newcor)

9000

In [74]:
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary = True)
tfidf_matrix = tfidf.fit_transform(newcor)

In [75]:
tfidf_matrix

<9000x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 681270 stored elements in Compressed Sparse Row format>

In [484]:

# idf = tfidf.idf_
# print (tfidf.get_feature_names(), idf)

In [485]:
# corpus = [
#     tokenize(df[0][0]),
#     tokenize(df[0][1]),
#     tokenize(df[0][2])
# ]

In [76]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 300)
x_train = tfidf_matrix.toarray()
x_train = pca.fit_transform(x_train)
y_train = []

for i in range(9000):
    y_train.append(train[i][1])
y_train = np.array(y_train)

In [77]:
print(x_train.shape)
# tfidf_matrix.toarray().shape
from sklearn.metrics import f1_score

(9000, 300)


In [80]:
from keras.utils import np_utils
tfidf_matrix = tfidf.transform(test)
# print(tfidf_matrix)
x_test = pca.transform(tfidf_matrix.toarray())
y_test = []

for i in range(9000,10000):
    y_test.append(train[i][1])
y_test = np.array(y_test)

# predictions = classifier.predict(x_test)
print(x_train.shape)
y_train.reshape(9000, 1)
y_train = np_utils.to_categorical(y_train)
y_train = y_train[:, 1:]
print(y_train)


(9000, 300)
[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.]
 ..., 
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.]]


In [81]:
# x_train.mean()
# predictions
f1_score(y_test, predictions, average='macro')

ValueError: Found input variables with inconsistent numbers of samples: [1000, 5000]

In [138]:
predictions = linearClassifier.predict(x_test)
predictions
f1_score(y_test, predictions, average='macro')

/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.18891432308698497

In [98]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
from keras.utils import np_utils
from keras.layers import Conv1D, MaxPooling1D, Flatten

# from sklearn.decomposition import PCA

# pca = PCA(n_components = 300)
# x_train = tfidf_matrix.toarray()
# x_train = pca.fit_transform(x_train)
# y_train = []

# for i in range(3500):
#     y_train.append(train[i][1])
# y_train = np.array(y_train)

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))
rate = 0.3

# def get_feedforward_nn():
#   input1 = Input(shape=(300,))
#   x = Dense(300, activation='relu')(input1)
#   x = Dropout(rate)(x)
#   x = Dense(128, activation='relu')(x)
#   x = Dropout(rate)(x)  
#   out = Dense(1, activation='relu')(x)

#   model = Model(inputs=input1, outputs=out)
#   model.compile(optimizer=Adam(),
#                 loss='mean_squared_error',
#                 metrics=[f1])
#   return model

import keras
from keras.models import Sequential
from keras.layers import Dense


# Initialising the ANN
classifier = Sequential()

#CNN
classifier.add(Conv1D(nb_filter = 128, filter_length = 3, input_shape = (300,1), padding = "same", activation = 'relu'))
classifier.add(MaxPooling1D())
classifier.add(Dropout(0.3))

# classifier.add(Conv1D(nb_filter = 128, filter_length = 3, padding = "same", activation = 'relu'))
# classifier.add(MaxPooling1D())
# classifier.add(Dropout(0.3))
classifier.add(Flatten())

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 300, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.3))

# Adding the second hidden layer
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.3))

# Adding the output layer
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [])

# for c in y_train:
#     if c not in {1, 2, 3, 4, 5}:
#         print(y)
classifier.fit(np.expand_dims(x_train, axis = 2), y_train, batch_size = 256, epochs = 25)

/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:77: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(300, 1), padding="same", activation="relu", filters=128, kernel_size=3)`
/Users/ekkalakleelasornchai/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:81: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(padding="same", activation="relu", filters=256, kernel_size=3)`


Epoch 1/25
9000/9000 [==============================] - 53s 6ms/step - loss: 1.3187 - f1: nan
Epoch 2/25
9000/9000 [==============================] - 52s 6ms/step - loss: 1.2600 - f1: nan
Epoch 3/25
9000/9000 [==============================] - 54s 6ms/step - loss: 1.2598 - f1: nan
Epoch 4/25
9000/9000 [==============================] - 51s 6ms/step - loss: 1.2574 - f1: nan
Epoch 5/25
9000/9000 [==============================] - 51s 6ms/step - loss: 1.2529 - f1: 0.0851
Epoch 6/25
9000/9000 [==============================] - 50s 6ms/step - loss: 1.2411 - f1: 0.1388
Epoch 7/25
9000/9000 [==============================] - 50s 6ms/step - loss: 1.2243 - f1: 0.1993
Epoch 8/25
9000/9000 [==============================] - 51s 6ms/step - loss: 1.2044 - f1: 0.2314
Epoch 9/25
9000/9000 [==============================] - 51s 6ms/step - loss: 1.1887 - f1: 0.2587
Epoch 10/25
9000/9000 [==============================] - 52s 6ms/step - loss: 1.1771 - f1: 0.2667
Epoch 11/25
9000/9000 [==================

In [99]:
res = classifier.predict(np.expand_dims(x_test, axis = 2))
res

array([[  9.29432514e-04,   6.35982081e-02,   2.11642474e-01,
          6.41497493e-01,   8.23323652e-02],
       [  9.32448966e-05,   4.47179162e-04,   2.73600370e-02,
          5.26446521e-01,   4.45653021e-01],
       [  7.62091545e-08,   1.00179532e-05,   1.54845744e-01,
          7.98186660e-01,   4.69575226e-02],
       ..., 
       [  1.69451520e-01,   1.61483496e-01,   2.22016156e-01,
          3.19906682e-01,   1.27142191e-01],
       [  3.84328817e-03,   5.98639771e-02,   3.52832198e-01,
          5.03494799e-01,   7.99657181e-02],
       [  1.03360685e-02,   6.86076237e-03,   1.38379335e-01,
          6.41734004e-01,   2.02689856e-01]], dtype=float32)

In [101]:
predictions = []
for i in range(1000):
    predictions.append(np.argmax(res[i]) + 1)
# print(predictions)
from sklearn.metrics import f1_score,precision_score,recall_score
f1_score(y_test,predictions, average='micro')
# len(predictions)
# y_test.shape

0.48799999999999999

In [451]:
from keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from keras import backend as K
K.clear_session()

# Path to save model parameters
weight_path_feedforward_nn='~/workspace/model/model_weight_feedforward_nn2.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.
callbacks_list_feedforward_nn = [
        TensorBoard(log_dir='~/workspace/model/Graph/ff2', histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

print(x_train.shape)
verbose = 1
model_feedforward_nn = get_feedforward_nn()
train_params = [(50, 64)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_feedforward_nn.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_test, y_test))

(3500, 300)
train with 50 epochs and 64 batch size


ValueError: Error when checking target: expected dense_3 to have shape (None, 1) but got array with shape (3500, 5)

In [65]:
# from keras.layers import Dropout

# def get_nn_with_dropout():
    
#     rate = 0.1
    
#     input1 = Input(shape=(21,))
#     x = Dense(100, activation='relu')(input1)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(rate)(x)
#     out = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                 loss='binary_crossentropy',
#                 metrics=['acc'])
#     return model

from sklearn.metrics import f1_score,precision_score,recall_score

################################################################################
# Write a function to evaluate your model. Your function must make prediction  #
# using the input model and return f-score, precision, and recall of the model.#
# You can make predictions by calling model.predict().                         #
################################################################################
def evaluate(x_test, y_test, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)

#     #map probability to class
#     prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
#     y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
#     for i in range(len(y_pred)):
#         if y_pred[i] - int(y_pred[i]) >= 0.5:
#             y_pred[i] = int(y_pred[i]) + 1
#         else:
#             y_pred[i] = int(y_pred[i])
    print(y_pred.T[0])
    print(y_test)
    f1score = f1_score(y_test,y_pred.T[0], average='macro')
    precision = precision_score(y_test,y_pred,average='macro')
    recall = recall_score(y_test,y_pred,average='macro')
    return f1score, precision, recall

In [66]:
# print(x_test.shape)
# y_pred = classifier.predict(x_test)
evaluate(x_test, y_test, classifier)

ValueError: Error when checking : expected conv1d_15_input to have 3 dimensions, but got array with shape (1000, 300)